In [3]:
import numpy as np
import pandas as pd
import json
import re

In [4]:
with open('data/daterangegovinfo02.json', 'r') as fl:
    govinfo_dct = json.loads(fl.read())

In [5]:
text_arr = [el['transcript'] for el in govinfo_dct]

clean_text_arr = []
CLEANR = re.compile('<.*?>')
CLEANR2 = re.compile('\[.*?\]')
CLEANR3 = re.compile("\s+")

for text in text_arr:
    temp = re.sub(CLEANR, '', text)
    temp = re.sub(CLEANR2, '', temp)
    temp = temp.replace('\n', ' ')
    temp = re.sub(CLEANR3, ' ', temp).strip()
    clean_text_arr.append(temp)

text_arr = clean_text_arr.copy()
del clean_text_arr
print(text_arr[0])

Congressional Record, Volume 168 Issue 2 (Tuesday, January 4, 2022) From the Congressional Record Online through the Government Publishing Office Remembering Johnny Isakson Mr. THUNE. Mr. President, before I begin, I want to express my deep sorrow at the death of former Senator Johnny Isakson. I served with Johnny for a long time, not just here in the Senate, where we came in together as freshman Members in 2005, but also in the House of Representatives, and I was honored to call him my friend. Johnny was a tremendously effective legislator and, in particular, a champion for veterans and a model of decency and graciousness. When I came down to the floor to discuss his retirement 2 years ago, I read a quote from a politics professor in Georgia that I thought really captured Johnny. I think so still. Here is what that professor had to say: As a political science professor and an administrator, I'm often asked by students if good people can serve in government and keep their integrity. Jo

In [43]:
rolldf = pd.read_csv('data/HSall_rollcalls.csv')
bills = list(set(rolldf['bill_number']))
bills[100]

/tmp/ipykernel_13051/2705642688.py:1: DtypeWarning: Columns (14,15,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  rolldf = pd.read_csv('data/HSall_rollcalls.csv')


'HR4200'

In [44]:
rolldf.head()

,congress,chamber,rollnumber,date,session,clerk_rollnumber,yea_count,nay_count,nominate_mid_1,nominate_mid_2,nominate_spread_1,nominate_spread_2,nominate_log_likelihood,bill_number,vote_result,vote_desc,vote_question,dtl_desc
0,1,House,1,1789-05-16,NaN,NaN,41,8,0.484,-0.872,0.229,-0.378,-13.144,HR2,NaN,NaN,NaN,"TO AMEND H.R. 2, (1 STAT. 24, APP. 7-4-1789), ..."
1,1,House,2,1789-05-22,NaN,NaN,36,1,0.256,0.967,-0.090,1.791,-1.900,NaN,NaN,NaN,NaN,TO PASS THE RESOLUTION THAT IT APPEARS TO THIS...
2,1,House,3,1789-06-22,NaN,NaN,30,18,-0.394,-0.454,-0.176,0.922,-22.048,HR8,NaN,NaN,NaN,"TO AMEND H.R. 8, (1 STAT. 28, APP. 7-27-1789),..."
3,1,House,4,1789-06-22,NaN,NaN,31,19,-0.194,-0.981,0.081,-0.327,-30.355,HR8,NaN,NaN,NaN,"TO AMEND H.R. 8, SO AS TO ELIMINATE THE PRESID..."
4,1,House,5,1789-06-24,NaN,NaN,29,22,-0.197,-0.266,-0.168,0.984,-22.221,HR8,NaN,NaN,NaN,TO PASS H.R. 8.


In [62]:
def get_bill_data(bill):
    if bill not in set(rolldf['bill_number']):
        return None
    return rolldf[rolldf['bill_number'] == bill]

In [88]:
with open("./congress/mentions_of_bills.json") as f:
	data = json.load(f)

bills = set()
bill_keys = []
data_arr = []

BILL_REGEX = re.compile(r"(H\.R\. \d+|H\. Res\. \d+|S\. \d+|S\. Res\. \d+)")
BILL_KEY = re.compile(r"[\s\.]")
for item in data:
	b = (re.search(BILL_REGEX, item['transcript']).group(1))
	bkey = re.sub(BILL_KEY, "", b)
	bdata = get_bill_data(bkey)
	if bdata is not None:
		data_arr.append([bkey, item['transcript'], int(max(bdata['yea_count']))])

In [89]:
print(data_arr[100][2])

220


In [90]:
len(data_arr)

5481